In [1]:
!git clone https://github.com/sfarrukhm/making_models_efficient.git
import os
os.chdir("/kaggle/working/making_models_efficient/")

Cloning into 'making_models_efficient'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 142 (delta 40), reused 32 (delta 9), pack-reused 57 (from 1)
Receiving objects: 100% (142/142), 88.06 MiB | 34.03 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
import warnings
warnings.filterwarnings("ignore")

### Create test data loader

In [3]:
from utils.data_utils import make_torch_dataset_from_image_folder
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
test_path = "/kaggle/input/intel-image-classification/seg_test/seg_test"

w,h=100,100
global_transforms = transforms.Compose([
  transforms.Resize((w, h)),
  # transforms.Grayscale(),
  transforms.ToTensor()
])


test_data=make_torch_dataset_from_image_folder(root=test_path,transform=global_transforms)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)


class_names = test_data.classes
class_num = len(class_names)
class_names, class_num

(['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'], 6)

### Load Models

In [4]:
teacher_path="models/checkpoints/teacher.pth"
student_path="models/checkpoints/student.pth"

In [5]:
import sys
sys.path.append('../')  # or whatever relative path goes up to the parent of `utils`

import torch
from models.student import StudentVisionEagle
from models.teacher import VisionEagle

# from utils.model_utils import load_model

# load_model(model_class, checkpoint_path, model_name, device='cuda' if torch.cuda.is_available() else 'cpu'):
student_model=StudentVisionEagle()
teacher_model=VisionEagle()

student_model.load_state_dict(torch.load("models/checkpoints/student.pth",map_location=torch.device('cpu') ))
teacher_model.load_state_dict(torch.load("models/checkpoints/teacher.pth",map_location=torch.device('cpu') ))


device="cuda" if torch.cuda.is_available() else "cpu"

teacher_model = teacher_model.to(device)
student_model = student_model.to(device)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 147MB/s]


### Evaluate

In [6]:
from utils.evaluate_trained_models import evaluate_accuracy, measure_latency

# Evaluate accuracy
teacher_acc = evaluate_accuracy(teacher_model, test_dataloader, device)
student_acc = evaluate_accuracy(student_model, test_dataloader, device)

# Evaluate latency
teacher_latency = measure_latency(teacher_model, device=device)
student_latency = measure_latency(student_model, device=device)

# Print comparison
print("\n📊 KD Evaluation Summary:")
print(f"Teacher Accuracy : {teacher_acc*100:.2f}%")
print(f"Student Accuracy : {student_acc*100:.2f}%")
print(f"Teacher Latency  : {teacher_latency:.2f} ms/image")
print(f"Student Latency  : {student_latency:.2f} ms/image")

Evaluating Accuracy: 100%|██████████| 47/47 [00:32<00:00,  1.44it/s]



📊 KD Evaluation Summary:
Teacher Accuracy : 89.50%
Student Accuracy : 81.13%
Teacher Latency  : 42.49 ms/image
Student Latency  : 10.46 ms/image
